In [1]:
#load relevant packages

from math import cos, asin, sqrt, pi
import pandas as pd
import numpy as np
import json
#from tqdm import tqdm
#tqdm.pandas()
import sqlalchemy
import psycopg2

In [3]:
# Business json einlesen
data_file = open("/home/sync/yelp/yelp_academic_dataset_business.json")
data = []
for line in data_file:
    data.append(json.loads(line))


In [4]:
restaurant_df = pd.DataFrame(data)
data_file.close()
#drop restaurants with false geo data
restaurant_df.drop(restaurant_df[restaurant_df.longitude > 0].index,inplace = True)
#edit df
restaurant_df.drop(columns =['hours', 'attributes'], inplace =True)

#filter for only open restaurants
#restaurant_df = restaurant_df[restaurant_df['is_open']==1]

#filter for only restaurants
restaurant_df = restaurant_df[restaurant_df.categories.str.contains("Restaurant")==True]
#drop restaurant which is the only one in state 'ABE'
restaurant_df.drop(restaurant_df[restaurant_df['business_id'] == 'WaD_wuvvDFPTrz8ouDPtAw' ].index, inplace = True)

In [12]:
categories_df = pd.DataFrame({"categories":list(set(restaurant_df.categories.str.split(pat=",").explode()))})
categories_df['categories'] = categories_df['categories'].str.strip()
categories_df = categories_df.drop_duplicates()
categories_df  = categories_df.reset_index(drop=True)
categories_df['categorie_id'] = categories_df.index + 1
categories_df

,categories,categorie_id
0,Sporting Goods,1
1,French,2
2,Barbeque,3
3,Bar Crawl,4
4,Mobile Phone Accessories,5
...,...,...
723,Eyebrow Services,724
724,Yoga,725
725,Skating Rinks,726
726,Hair Loss Centers,727


In [23]:
engine = sqlalchemy.create_engine('')
categories_df.to_sql('categorie', engine, if_exists='append', index = False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_categorie"
DETAIL:  Key (categorie_id)=(1) already exists.

[SQL: INSERT INTO categorie (categories, categorie_id) VALUES (%(categories)s, %(categorie_id)s)]
[parameters: ({'categories': 'Sporting Goods', 'categorie_id': 1}, {'categories': 'French', 'categorie_id': 2}, {'categories': 'Barbeque', 'categorie_id': 3}, {'categories': 'Bar Crawl', 'categorie_id': 4}, {'categories': 'Mobile Phone Accessories', 'categorie_id': 5}, {'categories': 'Canadian (New)', 'categorie_id': 6}, {'categories': 'Mountain Biking', 'categorie_id': 7}, {'categories': 'Bingo Halls', 'categorie_id': 8}  ... displaying 10 of 728 total bound parameter sets ...  {'categories': 'Hair Loss Centers', 'categorie_id': 727}, {'categories': 'Eyelash Service', 'categorie_id': 728})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [24]:
restaurant_df_tidy = restaurant_df[['business_id', 'categories']]
restaurant_df_tidy['categories'] = restaurant_df_tidy.categories.str.split(pat=",")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
restaurant_df_tidy

,business_id,categories
0,6iYb2HFDywm3zjuRg0shjw,"[Gastropubs, Food, Beer Gardens, Restaurant..."
1,tCbdrRPZA0oiIYSmHG3J0w,"[Salad, Soup, Sandwiches, Delis, Restauran..."
5,D4JtQNTI4X3KcbzacDJsMw,"[Restaurants, Thai]"
7,jFYIsSb7r1QeESVUnXPHBw,"[Breakfast & Brunch, Restaurants]"
12,HPA_qyMEddpAEtFof02ixg,"[Food, Pizza, Restaurants]"
...,...,...
160573,Zl6SUy6x9jqjRu2HbtEO6A,"[American (Traditional), Desserts, American ..."
160574,Q78fYV6B6P6GmX07YVgi4g,"[Mexican, Bars, Nightlife, Tex-Mex, Restau..."
160575,uXdQkuEtvLAzfc3MsO-sTQ,"[Restaurants, Mexican, Latin American]"
160583,GB75wPibj3IjNauaoCxyGA,"[Cuban, Sandwiches, Restaurants, Cafes]"


In [106]:
res = restaurant_df_tidy.explode('categories').reset_index(drop=True)
res['categories'] = res['categories'].str.strip()
res = res.rename(columns={"categories": "categorie_id"})

In [29]:

res = restaurant_df_tidy.explode('categories').reset_index(drop=True)
res['categories'] = res['categories'].str.strip()
res = res.rename(columns={"categories": "categorie_id"})

res

,business_id,categorie_id
0,6iYb2HFDywm3zjuRg0shjw,Gastropubs
1,6iYb2HFDywm3zjuRg0shjw,Food
2,6iYb2HFDywm3zjuRg0shjw,Beer Gardens
3,6iYb2HFDywm3zjuRg0shjw,Restaurants
4,6iYb2HFDywm3zjuRg0shjw,Bars
...,...,...
237678,ngmLL5Y5OT-bYHKU0kKrYA,Restaurants
237679,ngmLL5Y5OT-bYHKU0kKrYA,Middle Eastern
237680,ngmLL5Y5OT-bYHKU0kKrYA,Mediterranean
237681,ngmLL5Y5OT-bYHKU0kKrYA,Persian/Iranian


In [30]:
categories_dict = dict(zip(categories_df.categories, categories_df.categorie_id))

In [31]:
categories_dict

{'Sporting Goods': 1,
 'French': 2,
 'Barbeque': 3,
 'Bar Crawl': 4,
 'Mobile Phone Accessories': 5,
 'Canadian (New)': 6,
 'Mountain Biking': 7,
 'Bingo Halls': 8,
 'Cuban': 9,
 'Personal Chefs': 10,
 'Persian/Iranian': 11,
 'Tabletop Games': 12,
 'Convenience Stores': 13,
 'Marketing': 14,
 'Mediterranean': 15,
 'American (New)': 16,
 'Pizza': 17,
 'Whiskey Bars': 18,
 'Real Estate Agents': 19,
 'Service Stations': 20,
 'Beaches': 21,
 'Gas Stations': 22,
 'Music & DVDs': 23,
 'Professional Services': 24,
 'Swimming Pools': 25,
 'Cosmetics & Beauty Supply': 26,
 'Dance Studios': 27,
 'Dry Cleaning & Laundry': 28,
 'Parks': 29,
 'Poke': 30,
 'Orthodontists': 31,
 'Used': 32,
 'Teppanyaki': 33,
 'Soul Food': 34,
 'Bangladeshi': 35,
 'Painters': 36,
 'Property Management': 37,
 'Veterinarians': 38,
 'Mini Golf': 39,
 'Pasta Shops': 40,
 'Shanghainese': 41,
 'Wraps': 42,
 'Ferries': 43,
 'Shoe Stores': 44,
 'Dance Clubs': 45,
 'Pet Sitting': 46,
 'Barbers': 47,
 'Hookah Bars': 48,
 'Hima

In [32]:
res['categorie_id'] = res['categorie_id'].apply(lambda x: categories_dict[x])

In [33]:
res

,business_id,categorie_id
0,6iYb2HFDywm3zjuRg0shjw,374
1,6iYb2HFDywm3zjuRg0shjw,317
2,6iYb2HFDywm3zjuRg0shjw,363
3,6iYb2HFDywm3zjuRg0shjw,329
4,6iYb2HFDywm3zjuRg0shjw,529
...,...,...
237678,ngmLL5Y5OT-bYHKU0kKrYA,329
237679,ngmLL5Y5OT-bYHKU0kKrYA,68
237680,ngmLL5Y5OT-bYHKU0kKrYA,15
237681,ngmLL5Y5OT-bYHKU0kKrYA,11


In [111]:
res = res.drop_duplicates()

In [112]:
res.to_sql('has_categorie', engine, if_exists='append', index = False)